TO DO :
1.Importing dataset (done)
2.Data preprocessing :
        imputing (done)
        normalization  (done)
3.Feature selection  (done)
4.Train models and check performance of each :
        Logistic Regression (done)
        SVM    (done)
        Random Forest Classifier    (done)
5.Saving the model with highest accuracy    (done)
6.Loading the model from disk    (done)
7.Predict probability for new input point   (done)

In [3]:
#1)IMPORTING DATASET
import pandas as pd
dataFrame = pd.read_excel(r'C:\Users\Urooj\Desktop\Project material\data.xlsm',sheet_name='Full_new')
dataFrame=dataFrame.drop(["Sl. No","Patient File No."],axis=1)  #removing unneccesary columns

In [4]:
#2)DATA PREPROCESSING : (i)IMPUTATION (ii) NORMALIZATION(??why)

#(i)IMPUTATION: KNNImputer

#check for number of missing values in each column before imputation
print("--------BEFORE IMPUTATION--------")
print(dataFrame.isnull().sum())

#perform imputation
from sklearn.impute import KNNImputer
imputer = KNNImputer(n_neighbors=5)
dataFrame = pd.DataFrame(imputer.fit_transform(dataFrame),columns=dataFrame.columns)

#check for number of missing values in each column after imputation
print("--------AFTER IMPUTATION--------")
print(dataFrame.isnull().sum())

#(ii)NORMALIZATION: ??

#separating feature vectors from output labels
X = dataFrame.drop(['PCOS (Y/N)'],axis=1)
y = dataFrame['PCOS (Y/N)']

#getting the feature names
features = pd.DataFrame(X.columns)    

#performing normalization
from sklearn.preprocessing import MinMaxScaler
X = pd.DataFrame(MinMaxScaler().fit_transform(X))

#reattaching the column names to the X dataframe
X.columns = features.loc[:,0]

--------BEFORE IMPUTATION--------
PCOS (Y/N)                0
 Age (yrs)                0
Weight (Kg)               0
Height(Cm)                0
BMI                       0
Blood Group               0
Pulse rate(bpm)           0
RR (breaths/min)          0
Hb(g/dl)                  0
Cycle(R/I)                0
Cycle length(days)        0
Marraige Status (Yrs)     1
Pregnant(Y/N)             0
No. of aborptions         0
  I   beta-HCG(mIU/mL)    0
II    beta-HCG(mIU/mL)    0
FSH(mIU/mL)               0
LH(mIU/mL)                0
FSH/LH                    0
Hip(inch)                 0
Waist(inch)               0
Waist:Hip Ratio           0
TSH (mIU/L)               0
AMH(ng/mL)                1
PRL(ng/mL)                0
Vit D3 (ng/mL)            0
PRG(ng/mL)                0
RBS(mg/dl)                0
Weight gain(Y/N)          0
hair growth(Y/N)          0
Skin darkening (Y/N)      0
Hair loss(Y/N)            0
Pimples(Y/N)              0
Fast food (Y/N)           1
Reg.Exercise(Y

In [10]:
#3)FEATURE SELECTION:   ??
from sklearn.feature_selection import SelectKBest, chi2    #univariate feature selection

#instantiating SelectKBest to determine 20 best features
selector = SelectKBest(score_func=chi2,k=20)
scores = pd.DataFrame(selector.fit(X,y).scores_)

#creating a dataframe to list feature names along with scores to find the best ones
feature_scores = pd.concat([features,scores],axis=1)
feature_scores.columns = ['Feature_Name','Score']

#dropping unattainable features(requiring ultrasound) from this list
feature_scores = feature_scores.drop([36,37,38],axis=0)

#printing best features
print(feature_scores.nlargest(20,'Score'))

#getting best feature names
best_feature_names = feature_scores.nlargest(20,'Score')['Feature_Name'].tolist()
#input for later use is to be provided in this order only

#altering X so that it contains only best features
X = X.loc[:,best_feature_names]

             Feature_Name      Score
29   Skin darkening (Y/N)  84.870716
28       hair growth(Y/N)  84.854623
27       Weight gain(Y/N)  65.554147
8              Cycle(R/I)  42.176860
32        Fast food (Y/N)  37.462765
31           Pimples(Y/N)  22.587803
30         Hair loss(Y/N)   8.846546
22             AMH(ng/mL)   3.580991
33      Reg.Exercise(Y/N)   1.737463
24         Vit D3 (ng/mL)   1.575758
0               Age (yrs)   1.402764
1             Weight (Kg)   1.337848
16             LH(mIU/mL)   1.271769
3                     BMI   1.123187
18              Hip(inch)   0.848799
19            Waist(inch)   0.836235
10  Marraige Status (Yrs)   0.670866
9      Cycle length(days)   0.645862
12      No. of aborptions   0.586935
15            FSH(mIU/mL)   0.321571


In [12]:
#4)TRAIN MODELS: 

#splitting the dataset into training and testing datasets
import numpy as np
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,train_size=0.7)

#(i)Logistic Regression :
from sklearn.linear_model import LogisticRegression
np.random.seed(1)  #just to make sure random_state of LogisticRegression() constructor works
LR = LogisticRegression(random_state=1) #random_state is used to set seed for random number generator so that we get same results across multiple runs
LR.fit(X_train,y_train)
print(LR.score(X_test,y_test)) #model.score() gives accuracy score(#correct_preds/#all_preds). Calculates predictions for X_test under the hood and compares to y_test

#(ii)SVM :
from sklearn import svm
SVM = svm.SVC(C=20,kernel='linear')
SVM.fit(X_train,y_train)
print(SVM.score(X_test,y_test))

#(iii)Random Forest Classifier :
from sklearn.ensemble import RandomForestClassifier
RFC = RandomForestClassifier()
RFC.fit(X_train,y_train)
print(RFC.score(X_test,y_test))

0.8588957055214724
0.8282208588957055
0.8159509202453987


In [16]:
#5)SAVING MODEL WITH HIGHEST ACCURACY :

#Logistic regression model is saved as it has the highest accuracy
import pickle
model_filename = "PCOS_SVM.sav"
pickle.dump(LR, open(model_filename, 'wb'))

In [17]:
#6)LOADING THE MODEL FROM DISK : 
loaded_model = pickle.load(open("PCOS_SVM.sav",'rb'))
print(loaded_model.score(X_test, y_test))

0.8588957055214724


In [19]:
#7)PREDICT PROBABILITY FOR NEW INPUT POINT :
print(X_test.iloc[0])
#creating a new data point (actually take this from user)
inp = X_test.iloc[0].tolist()
#getting predicted probability for this input point
prob_distribution = loaded_model.predict_proba([inp]) #this func takes as argument a list of data points and returns an array where each element is a probability distribution(list of 2 elements) for each point
prob_having_PCOD = prob_distribution[0][1] #the second element in each probability dist. gives probability of point belonging to the class 1
print("%f%% chance of having PCOD."%(prob_having_PCOD*100))

0
Skin darkening (Y/N)     0.000000
hair growth(Y/N)         0.000000
Weight gain(Y/N)         0.000000
Cycle(R/I)               0.000000
Fast food (Y/N)          1.000000
Pimples(Y/N)             0.000000
Hair loss(Y/N)           0.000000
AMH(ng/mL)               0.254932
Reg.Exercise(Y/N)        1.000000
Vit D3 (ng/mL)           0.003924
 Age (yrs)               0.250000
Weight (Kg)              0.272727
LH(mIU/mL)               0.001095
BMI                      0.403371
Hip(inch)                0.454545
Waist(inch)              0.347826
Marraige Status (Yrs)    0.166667
Cycle length(days)       0.416667
No. of aborptions        0.000000
FSH(mIU/mL)              0.000942
Name: 444, dtype: float64
9.485247% chance of having PCOD.
